In [1]:
import requests
from pprint import pprint
from IPython.display import display, HTML, clear_output
import time

# 得到A股基本信息参考表 a_refer
a_list = ['name', 
            'open_price', 
            'yesterday_price', 
            'current_price', 
            'max', 'min', 
            'buy1', 'sell1', 
            'volume', # 成交量
            'amount', # 成交额
            'buy1_volume', # 买1申请量
            'buy1', # 买1价格
            'buy2_volume', 'buy2', 'buy3_volume', 'buy3', 'buy4_volume', 'buy4', 'buy5_volume', 'buy5', 
            'sell1_volume', # 卖1申请量
            'sell1', # 卖1价格
            'sell2_volume', 'sell2', 'sell3_volume', 'sell3', 'sell4_volume', 'sell4', 'sell5_volume', 'sell5',
            'date', 'time'
            ]
a_refer = {}
for i in range(len(a_list)):
    a_refer[a_list[i]] = i

# a_refer

# 得到港股基本信息参考表 rt_refer
rt_list = ['name_en', 'name',
           'open_price', 'yesterday_price', 'max', 'min', 'current_price', 
           'up_price', 'up_rate', # 涨跌价与涨跌百分比幅度
           'buy1', 'sell1', 'volume', 'amount',
           'pe', 'dividend_yield', # 市盈率、周息率
           'max_of_52weeks', 'min_of_52weeks',
           'date', 'time'
          ]

rt_refer = {}
for i in range(len(rt_list)):
    rt_refer[rt_list[i]] = i
# rt_refer

# 得到美股基本信息参考表 gb_refer
gb_list = ['name', 'current_price', 'up_rate', # 涨跌百分比幅度
           'datetime', # 日期和时间
           'up_price', # 上涨价格
           'open_price', 'max', 'min', 'max_of_52weeks', 'min_of_52weeks',
           'volume', 'volume_of_10days', 'maket_value',
           'earn_per_stock', 'pe', 'unknown1', 'beta', 'uknown2', 'uknown3',
           'capital_stock', 'unknown4', 
           'close_price', 'uknown5', 'uknown6', 'known7', 
           'UTC_time', 'yesterday_price', 'unkown8' 
          ]

gb_refer = {}
for i in range(len(gb_list)):
    gb_refer[gb_list[i]] = i
# gb_refer

stock_list = {
#     'gb_tal': '好未来',
#     'gb_$dji': '道琼斯',
    'rt_hkHSI': '恒生指数',
    'sh000001': '上证指数',
    'sh000016': '上证50',
    'sh000300': '沪深300',
    'sh000827': '中证环保',
    'sh000905': '中证500',
    'sh000991': '全指医药',
    'sh513030': '德国30',
    'sh513050': '中概互联',
    'sz159934': '黄金ETF',
    'sz162411': '华宝油气',
    'sz399006': '创业板指',
    'sz399396': '国证食品',
    'sz399812': '养老产业',
    'sz399932': '中证消费',
    'sz399967': '中证军工',
    'sz399971': '中证传媒',
    'sz399975': '证券公司'
}

url = 'http://hq.sinajs.cn/list=' + ','.join([key for key in stock_list])

html_model ="""<div style="margin: 3px; overflow: hidden;">
    <div style="float:left;position:relative;">

        <div style="color:white;height:25px;width:75px;text-align: center;padding-left:2px;padding-right:2px;margin-right:2px;z-index:11;
        background-color:#e2d5d5;color:#000000;font-weight: bold;float:left;">{name}</div> <!-- 名称 -->
        <div style="color:white;height:25px;width:55px;text-align: center;padding-left:2px;padding-right:2px;margin-right:2px;z-index:11;
       background-color:#e2d5d5;color:#000000;font-weight: bold;float:left;">{code}</div> <!-- 代码 -->
        <div style="color:white;height:25px;width:75px;text-align: right;padding-left:2px;padding-right:2px;margin-right:2px;z-index:11;
        background-color:#adadad;color: #353030;float:left;">{yesterday_price}</div> <!-- 昨日收盘价 -->
        <div style="color:white;height:25px;width:75px;text-align: right;padding-left:2px;padding-right:2px;margin-right:2px;z-index:11;
        background-color:#6666ff;float:left;">{current_price}</div> <!-- 现价 -->
         <!-- 前5个糅合一起的区间 -->
    </div>

    <div style="float:left;position:relative;">
        <div style="color:{color};width:56px;height:25px;text-align: center;padding-left:2px;margin-right:2px;z-index:1;
        background-color:{flash_color};position:absolute;font-weight: bold;border: 1.2px solid {color};">{up_p_r}</div> 
                <!-- 涨跌幅color up -->


        <div style="margin-left:60px;width:{total_width}px;height:25px;z-index:1;
    background-image:linear-gradient(to right, #6666ff, white, {color});position:absolute;">
    <div style="float:left;line-height:25px;text-align:left;font-size:11px;color:white;">{min_p_r}</div>    
    <div style="float:right;line-height:25px;text-align:right;font-size:11px;color:white;">{max_p_r}</div>    
    </div>  <!-- 涨跌停展示区的宽度，内部显示最低价和最高价 width，min，max -->
    
        <div style="margin-left:{middle_px}px;width:2px;height:25px;z-index:4;
    background-color:#FFFFFF;position:absolute;"></div> <!-- 中间0%%处的小白条 -->


        <div style="margin-left:{bar_px}px;width:{bar_width}px;height:25px;z-index:2;
    background-color:#E7D9A0;position:absolute;"></div> <!-- 最高价和最低价的区间展示bar_left, bar_width -->

        <div style="margin-left:{recent_px[0]}px;width:5px;height:31px;z-index:3;
    background-color:#ebeb0e;position:absolute;top:-3px;opacity:0.2;border-right:2px outset blue;"></div>
        <div style="margin-left:{recent_px[1]}px;width:5px;height:31px;z-index:3;
    background-color:#ebeb0e;position:absolute;top:-3px;opacity:0.4;border-right:2px outset blue;"></div>
        <div style="margin-left:{recent_px[2]}px;width:5px;height:31px;z-index:3;
    background-color:#ebeb0e;position:absolute;top:-3px;opacity:0.5;border-right:2px outset blue;"></div>
        <div style="margin-left:{recent_px[3]}px;width:5px;height:31px;z-index:3;
    background-color:#ebeb0e;position:absolute;top:-3px;opacity:0.7;border-right:2px outset blue;"></div>
        <div style="margin-left:{recent_px[4]}px;width:5px;height:31px;z-index:5;
    background-color:#ebeb0e;position:absolute;top:-3px;opacity:1;border-right:2px outset blue;"></div>
    <!-- 位置坐标 5个 -->
    </div>
</div>""" # 需要参数如下
# name, code, yesterday_price, current_price, up(p/r), total_width, min,max(p/r), bar_start, bar_width
#        px_recent[0], px_recent[1], px_recent[2], px_recent[3], px_recent[4], 
#     需要区分rate/price: up,min,max， 所有结果均为string

stock_cache = { } # 用于存储当前获得所有信息的字典
for key in stock_list:
    stock_cache[key] = {'info':[], 'price_info': [], 'rate_info': [], 'recent_rates': [0, 0, 0, 0, 0]}


def update_stock(mark_rate = 1):
    r_list = requests.get(url).content.decode('gbk').split('\n')[:-1] # 除去最终的空行

    # stock_all_dic = {} # 用于更新最终的股票代码和名字的字典

    for r in r_list:
        r_sp = r[11:-1].split('=')
            # 除去头尾部信息，根据 '=' 切割得到代码和结果，再根据 ',' 切割得到结果

        code = r_sp[0]
        result_l = r_sp[1][1:-1].split(',') # 得到最终的每份信息

    #     print(result)
        refer = {} # 根据不同的代码选择不同的解构字典
        s_code = ''
        if code[:2] == 'rt':
            refer = rt_refer
            s_code = code.split('_')[-1]
        elif code[:2] == 'gb':
            refer = gb_refer
            s_code = code.split('_')[-1]
        else:
            refer = a_refer
            s_code = code[-6:]
        name = result_l[refer['name']]
        yesterday_price = float(result_l[refer['yesterday_price']])
        current_price = float(result_l[refer['current_price']])

        price_change = current_price - yesterday_price
        change_rate = price_change / yesterday_price
        
        min_price = float(result_l[refer['min']])
        max_price = float(result_l[refer['max']])
        min_rate = (min_price - yesterday_price) / yesterday_price
        max_rate = (max_price - yesterday_price) / yesterday_price
        
        
        stock_cache[code]['info'] = [name, s_code, 
                                     format_point(current_price, yesterday_price),
                                     format_point(current_price, current_price)
                                    ]
        stock_cache[code]['price_info'] = [
                                     format_point(current_price, price_change),
                                     format_point(current_price, min_price),
                                     format_point(current_price, max_price),
                                          ]
        stock_cache[code]['rate_info'] = [change_rate, 
                                          min_rate,
                                          max_rate,
                                          ]
        
        if mark_rate: # 如果需要打点，则栈式输入
            del stock_cache[code]['recent_rates'][0]
            stock_cache[code]['recent_rates'].append(change_rate)



    #     result_dic[code] = result_l[]

    #     stock_all_dic[code] = result_l[refer['name']] # 用于更新所有股票的name内容
    # pprint(stock_all_dic)

def format_point(price, float_num):
    return '%.2f' % float_num if price > 100 else '%.3f' % float_num

In [2]:
count = 0
start_px = 60
px_for_10percent = 200
while(True):
    output_price = ''
    output_rate = ''
    output_price_flash = ''
    output_rate_flash = ''
#     sortby = up_rate_reversed
    count = 0 if count == 30 else count + 1 # 20s 更新一次
    update_stock(mark_rate = 1 if count == 1 else 0)
    
    stock_cache_tuple = sorted(stock_cache.items(), key = lambda item: float(item[1]['rate_info'][0]))
    # 得到元组，每一个地方的第一位为code，后面是一个dict
    
    price_format_list = []
    rate_format_list = []
    
    for stock in stock_cache_tuple:
        color = 'green' if stock[1]['rate_info'][0] < 0 else 'red'
        delta_rate = stock[1]['recent_rates'][-1] - stock[1]['recent_rates'][-2]
        flash_color = ''
        if delta_rate > 0.0001:
            flash_color = 'red'
        elif delta_rate < -0.0001:
            flash_color = 'green'
        else:
            flash_color = 'white'
        
        format_dic = {'name': stock[1]['info'][0], 
                     'code' : stock[1]['info'][1],
                     'yesterday_price' : stock[1]['info'][2],
                     'current_price' : stock[1]['info'][3],
                      'total_width': '%d'%(2*px_for_10percent),
                      'middle_px': '%d' % (start_px + px_for_10percent),
                      'bar_px': '%.0f' % (start_px + (1+stock[1]['rate_info'][1]*10) * px_for_10percent),
          'bar_width': '%.0f' % ((stock[1]['rate_info'][2] - stock[1]['rate_info'][1]) *10 * px_for_10percent),
          'recent_px': ['%.0f' % (start_px + (1 + i * 10) * px_for_10percent) for i in stock[1]['recent_rates']],
                      
                     'color': color,
                     'flash_color': flash_color,
                      'up_p_r': stock[1]['price_info'][0],
                      'min_p_r': stock[1]['price_info'][1],
                      'max_p_r': stock[1]['price_info'][2],
                     }
        
        output_price_flash += html_model.format(**format_dic)
        format_dic['flash_color'] = 'white'
        output_price += html_model.format(**format_dic)
        
        format_dic.update({
            'flash_color': flash_color, 
            'up_p_r': '%.2f%%'%(100*stock[1]['rate_info'][0]),
            'min_p_r': '%.2f%%'%(stock[1]['rate_info'][1]*100),
            'max_p_r': '%.2f%%'%(stock[1]['rate_info'][2]*100),
        })
        output_rate_flash += html_model.format(**format_dic)
        format_dic['flash_color'] = 'white'
        output_rate += html_model.format(**format_dic)
        
   
    for i in range(2):
        clear_output()
        display(HTML(output_price_flash))
        time.sleep(0.3)
        clear_output()
        display(HTML(output_price))
        time.sleep(2)
        clear_output()
        display(HTML(output_rate_flash))
        time.sleep(0.3)
        clear_output()
        display(HTML(output_rate))
        time.sleep(3)
    
    

KeyboardInterrupt: 

In [4]:
stock_cache

{'rt_hkHSI': {'info': ['恒生指数', 'hkHSI', '30119.56', '29820.46'],
  'price_info': ['-299.10', '29783.67', '30184.61'],
  'rate_info': [-0.009930357878081872,
   -0.011151856506265673,
   0.00215979257863635],
  'recent_rates': [0, 0, 0, 0, 0]},
 'sh000001': {'info': ['上证指数', '000001', '3241.93', '3197.89'],
  'price_info': ['-44.04', '3188.12', '3254.13'],
  'rate_info': [-0.013583020410157623,
   -0.01659712012897012,
   0.0037619567283055416],
  'recent_rates': [0, 0, 0, 0, 0]},
 'sh000016': {'info': ['上证50', '000016', '2979.10', '2927.27'],
  'price_info': ['-51.83', '2920.19', '3000.10'],
  'rate_info': [-0.017398249839674612,
   -0.019772659413577646,
   0.007048004622875056],
  'recent_rates': [0, 0, 0, 0, 0]},
 'sh000300': {'info': ['沪深300', '000300', '4085.85', '4004.91'],
  'price_info': ['-80.94', '3991.99', '4097.12'],
  'rate_info': [-0.019808940471828802,
   -0.0229722258322449,
   0.0027602355154266885],
  'recent_rates': [0, 0, 0, 0, 0]},
 'sh000827': {'info': ['中证环保', '0

In [1]:
print(output_rate_flash)

NameError: name 'output_rate_flash' is not defined

In [94]:
# 获取数据的验证块
s = 'HSCEI,恒生中国企业指数,11771.591,11815.000,11774.591,11694.320,11737.960,-77.040,-0.650,0.000,0.000,13554505.728,0,0.000,0.000,12589.530,9761.600,2019/04/10,12:05:00,,,,,,'
sp = s.split(',')
for key in rt_refer:
    print('%-15s----|%s'% (key, sp[rt_refer[key]]))

name_en        ----|HSCEI
name_zh        ----|恒生中国企业指数
open_price     ----|11771.591
yesterday_price----|11815.000
max            ----|11774.591
min            ----|11694.320
current_price  ----|11737.960
up_price       ----|-77.040
up_rate        ----|-0.650
buy1           ----|0.000
sell1          ----|0.000
volume         ----|13554505.728
amount         ----|0
pe             ----|0.000
dividend_yield ----|0.000
max_of_52weeks ----|12589.530
min_of_52weeks ----|9761.600
date           ----|2019/04/10
time           ----|12:05:00


## 华宝油气

### 支撑位
- 0.5

- 0.46

### 压力位
- 0.73

- 0.63

# 网格点
- 0.44
- 0.50
- 0.54
- 0.58
- 0.62
- 0.71